## Information

[facenet-pytorch](https://github.com/timesler/facenet-pytorch)

[face-tracking](https://github.com/timesler/facenet-pytorch/blob/master/examples/face_tracking.ipynb)

## Packages

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
import numpy as np
import mmcv, cv2
from PIL import Image, ImageDraw
from IPython import display

## Further Steps

### Determine if an nvidia GPU is available¶


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))